# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [2]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("/Users/meichelyoingco/Desktop/Python-Api-challenge/output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,tazovsky,67.4667,78.7000,1.54,82,100,22.57,RU,1712894456
1,1,egilsstadir,65.2653,-14.3948,35.51,75,75,9.22,IS,1712894456
2,2,bredasdorp,-34.5322,20.0403,54.00,79,34,6.22,ZA,1712894456
3,3,grytviken,-54.2811,-36.5092,30.72,74,25,10.38,GS,1712894202
4,4,cannonvale,-20.2833,148.7000,74.59,75,89,16.06,AU,1712894457


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [35]:
%%capture --no-display

# Configure the map plot
city_map = city_data_df.hvplot.points("Lng", 
                                      "Lat", 
                                      geo = True,
                                      frame_width=800,
                                      frame_height=600,
                                      size = "Humidity",
                                      tiles = "EsriImagery",
                                      scale=0.5
                                     )

# Display the map
city_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [17]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather_df = city_data_df.loc[(city_data_df["Max Temp"] > 3)
                                       & (city_data_df["Wind Speed"] < 10)
                                      ]

# Drop any rows with null values
ideal_weather_df.dropna()

# Display sample data
ideal_weather_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
1,1,egilsstadir,65.2653,-14.3948,35.51,75,75,9.22,IS,1712894456
2,2,bredasdorp,-34.5322,20.0403,54.00,79,34,6.22,ZA,1712894456
7,7,papatowai,-46.5619,169.4708,48.69,67,85,4.88,NZ,1712894457
8,8,ushuaia,-54.8000,-68.3000,46.06,71,0,9.22,AR,1712894458
10,10,weno,7.4515,151.8468,87.75,79,100,9.22,FM,1712894458
...,...,...,...,...,...,...,...,...,...,...
576,576,sawai madhopur,25.9833,76.3667,90.86,25,97,7.92,IN,1712894485
578,578,darovskoy,58.7700,47.9564,43.34,84,100,8.86,RU,1712894544
579,579,fale old settlement,-9.3852,-171.2468,86.32,70,53,8.08,TK,1712894544
580,580,astorp,56.1357,12.9443,48.16,100,100,8.05,SE,1712894544


### Step 3: Create a new DataFrame called `hotel_df`.

In [18]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
1,egilsstadir,IS,65.2653,-14.3948,75,
2,bredasdorp,ZA,-34.5322,20.0403,79,
7,papatowai,NZ,-46.5619,169.4708,67,
8,ushuaia,AR,-54.8000,-68.3000,71,
10,weno,FM,7.4515,151.8468,79,
...,...,...,...,...,...,...
576,sawai madhopur,IN,25.9833,76.3667,25,
578,darovskoy,RU,58.7700,47.9564,84,
579,fale old settlement,TK,-9.3852,-171.2468,70,
580,astorp,SE,56.1357,12.9443,100,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [25]:
# Set parameters to search for a hotel
radius = 10000
params = {"categories": "accommodation.hotel",
          "apiKey":geoapify_key
         }

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    longitude = row["Lng"]
    latitude = row["Lat"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params = params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
egilsstadir - nearest hotel: Valaskjálf
bredasdorp - nearest hotel: Victoria Hotel
papatowai - nearest hotel: No hotel found
ushuaia - nearest hotel: Apart Hotel Aires del Beagle
weno - nearest hotel: High Tide Hotel
toamasina - nearest hotel: Androranga
gallup - nearest hotel: No hotel found
huarmey - nearest hotel: Hostal Santa Rosa
ixtapa - nearest hotel: Ma. Cristina
labasa - nearest hotel: Grand Eastern Hotel
albany - nearest hotel: No hotel found
ribeira brava - nearest hotel: Cheerfulway Bravamar Hotel
blackmans bay - nearest hotel: Villa Howden
kodiak - nearest hotel: No hotel found
west island - nearest hotel: Cocos Village Bungalows
spittal an der drau - nearest hotel: Erlebnis Post Hotel
bethel - nearest hotel: Hampton Inn Danbury
tongchuan - nearest hotel: No hotel found
caxias - nearest hotel: Hjem
beledweyne - nearest hotel: Hotel Medina
haiku-pauwela - nearest hotel: Inn At Mama's Fish House
kolonia - nearest hotel: Wasserturm Hotel Cologne
kalevala

,City,Country,Lat,Lng,Humidity,Hotel Name
1,egilsstadir,IS,65.2653,-14.3948,75,Valaskjálf
2,bredasdorp,ZA,-34.5322,20.0403,79,Victoria Hotel
7,papatowai,NZ,-46.5619,169.4708,67,No hotel found
8,ushuaia,AR,-54.8000,-68.3000,71,Apart Hotel Aires del Beagle
10,weno,FM,7.4515,151.8468,79,High Tide Hotel
...,...,...,...,...,...,...
576,sawai madhopur,IN,25.9833,76.3667,25,Ranthambore Tiger Valley
578,darovskoy,RU,58.7700,47.9564,84,Центральная
579,fale old settlement,TK,-9.3852,-171.2468,70,No hotel found
580,astorp,SE,56.1357,12.9443,100,Restaurang Hotel Rosenberg


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [37]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points("Lng", 
                                   "Lat", 
                                   geo = True,
                                   frame_width=800,
                                   frame_height=600,
                                   size = "Distance",
                                   tiles = "EsriImagery",
                                   scale=0.5
                                  )



# Display the map
hotel_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]